# Table 2.

Global diagnostics of in Hab1 and Hab2 simulations.

In [1]:
# Data analysis and viz libraries
import pandas as pd
import xarray as xr

In [2]:
# Local modules
import mypaths
from names import names
from calc import INTERPOLATOR, bond_albedo, greenhouse_effect, cre_toa, spatial_mean, upper_atm_vap_mean
# from load_thai import LOAD_CONF
from commons import MODELS

## Choose cases

In [3]:
THAI_cases = ["Hab1", "Hab2"]

## Loading the data

Load the time-averaged data previously preprocessed.

In [4]:
# Load data
datasets = {}  # Create an empty dictionary to store all data
# Loop over THAI cases (this loop can swapped places with the loop below if needed)
for THAI_case in THAI_cases:
    # for each of the THAI cases, create a nested directory for models
    datasets[THAI_case] = {}
    for model_key in MODELS.keys():
        datasets[THAI_case][model_key] = xr.open_dataset(
            mypaths.datadir
            / model_key
            / f"{THAI_case}_regridded_time_mean_{model_key}.nc"
        )
        # datasets[THAI_case][model_key] = LOAD_CONF[model_key]["loader"](THAI_case)
    datasets[THAI_case]["ExoCAM"][names.exocam.lev].attrs.update(units="hPa")
    datasets[THAI_case]["ROCKE3D"][names.rocke3d.pres].attrs.update(units="hPa")
    datasets[THAI_case]["ROCKE3D"][names.rocke3d.lev].attrs.update(
        positive="up", units="1"
    )

In [5]:
diags = {
    "cre_sw": {
        "func": lambda ds, model_key: cre_toa(ds, model_key, kind="sw"),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"$CRE_{SW}$",
    },
    "cre_lw": {
        "func": lambda ds, model_key: cre_toa(ds, model_key, kind="lw"),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"$CRE_{LW}$",
    },
    "cre_tot": {
        "func": lambda ds, model_key: cre_toa(ds, model_key, kind="total"),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"$CRE$",
    },
    "alb": {"func": bond_albedo, "fmt": lambda x: f"{x:.2f}", "title": r"$\alpha_p$"},
    "ghe_as": {
        "func": lambda ds, model_key, const: greenhouse_effect(
            ds, model_key, const, kind="all_sky"
        ),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"$G_{as}$",
    },
    "ghe_cs": {
        "func": lambda ds, model_key, const: greenhouse_effect(
            ds, model_key, const, kind="clear_sky"
        ),
        "fmt": lambda x: f"{x:.1f}",
        "title": r"$G_{cs}$",
    },
    "vmr_1hpa": {
        "func": lambda ds, model_key, const: upper_atm_vap_mean(ds, model_key, const, [100]),
        "fmt": lambda x: f"{x:.1e}",
        "title": r"$vmr_{1hPa}$",
    },
}

In [6]:
results = {}
for THAI_case in THAI_cases:
    if THAI_case.endswith("1"):
        import const_ben1_hab1 as const
    else:
        import const_ben2_hab2 as const
    dfs = {}
    for model_key in MODELS.keys():
        _data = {}
        for varkey, vardict in diags.items():
            if varkey in ["ghe_cs", "ghe_as", "vmr_1hpa"]:
                args = [const]
            else:
                args = []
            _data[vardict["title"]] = float(
                spatial_mean(
                    vardict["func"](datasets[THAI_case][model_key], model_key, *args),
                    names[model_key].x,
                    names[model_key].y,
                ).values
            )
        dfs[model_key] = pd.DataFrame(
            _data, index=pd.Index(name="GCM", data=[model_key])
        )
    results[THAI_case] = pd.concat(dfs, axis="index").droplevel(0).T
df = pd.concat(results).T

In [7]:
df

Hab1                                                        \
        $CRE_{SW}$ $CRE_{LW}$      $CRE$ $\alpha_p$   $G_{as}$  $G_{cs}$   
GCM                                                                        
ExoCAM  -37.324125  15.219436 -22.104689   0.241737  10.051711  5.598415   
LMDG    -21.520519   4.988985 -16.531534   0.218544   5.843514  4.590518   
ROCKE3D -44.545721   9.301546 -35.244175   0.274411   9.328334  6.661019   
UM      -48.096010  10.114035 -37.981976   0.281368   2.575571 -0.013824   

                            Hab2                                              \
         $vmr_{1hPa}$ $CRE_{SW}$ $CRE_{LW}$      $CRE$ $\alpha_p$   $G_{as}$   
GCM                                                                            
ExoCAM   4.603049e-07 -21.689450  13.124007  -8.565443   0.153784  52.833456   
LMDG     5.809857e-08 -31.820248   3.201599 -28.618649   0.212989  34.107985   
ROCKE3D  5.127907e-07 -34.961858   8.598856 -26.363002   0.187658  44.677686   
UM       5.050648e-07 -27.340366   8.303770 -19.036597   0.160655  39.087843   

                                 
          $G_{cs}$ $vmr_{1hPa}$  
GCM                              
ExoCAM   48.794599     0.000014  
LMDG     33.062221     0.000004  
ROCKE3D  41.927532     0.000045  
UM       36.498985     0.000009

In [8]:
formatters = {k: {j["title"]: j["fmt"] for j in diags.values()}[k[1]] for k in df.columns}
print(
    df.to_latex(
        formatters=formatters,
        column_format="l" + "c" * len(diags) * len(THAI_cases),
        escape=False,
    )
)

\begin{tabular}{lcccccccccccccc}
\toprule
{} & \multicolumn{7}{l}{Hab1} & \multicolumn{7}{l}{Hab2} \\
{} & $CRE_{SW}$ & $CRE_{LW}$ & $CRE$ & $\alpha_p$ & $G_{as}$ & $G_{cs}$ & $vmr_{1hPa}$ & $CRE_{SW}$ & $CRE_{LW}$ & $CRE$ & $\alpha_p$ & $G_{as}$ & $G_{cs}$ & $vmr_{1hPa}$ \\
GCM     &            &            &       &            &          &          &              &            &            &       &            &          &          &              \\
\midrule
ExoCAM  &      -37.3 &       15.2 & -22.1 &       0.24 &     10.1 &      5.6 &      4.6e-07 &      -21.7 &       13.1 &  -8.6 &       0.15 &     52.8 &     48.8 &      1.4e-05 \\
LMDG    &      -21.5 &        5.0 & -16.5 &       0.22 &      5.8 &      4.6 &      5.8e-08 &      -31.8 &        3.2 & -28.6 &       0.21 &     34.1 &     33.1 &      3.8e-06 \\
ROCKE3D &      -44.5 &        9.3 & -35.2 &       0.27 &      9.3 &      6.7 &      5.1e-07 &      -35.0 &        8.6 & -26.4 &       0.19 &     44.7 &     41.9 &      4.5e-05 \\